In [24]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import RandomNormal, HeNormal, GlorotNormal
import cv2
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.model_selection import train_test_split

In [2]:
#溶氧量點數
def DO_value(num):
  if (num >= 6.5):
    return 1
  elif(num >= 4.6):
    return 3
  elif(num >= 2.0):
    return 6
  else:
    return 10

#懸浮固體點數
def TSS_value(num):
  if (num <= 20.0):
    return 1
  elif(num <= 49.9):
    return 3
  elif(num <= 100):
    return 6
  else:
    return 10

#生化需氧量點數
def BOD_value(num):
  if (num <= 3.0):
    return 1
  elif(num <= 4.9):
    return 3
  elif(num <= 15.0):
    return 6
  else:
    return 10

#氨氣點數
def NH3N_value(num):
  if (num <= 0.5):
    return 1
  elif(num <= 0.99):
    return 3
  elif(num <= 3.0):
    return 6
  else:
    return 10

#計算RPI(River Pollution Index)
def cal_RPI(DO, TSS, BOD, NH3N):
  DO_num = DO_value(DO)
  TSS_num = TSS_value(TSS)
  BOD_num = BOD_value(BOD)
  NH3N_num = NH3N_value(NH3N)
  return (DO_num + TSS_num + BOD_num + NH3N_num)/4

In [3]:
# RPI = (DO + SS + BOD + NH3N) / 4
# NH3-N / NO3-N = K  -->  NH3-N = NO3-N * K

In [4]:
# def cal_RPI(DO, SS, BOD, NH3N):
#     return (DO + SS + BOD + NH3N) / 4

In [5]:
APRIL_RPI = cal_RPI(9.0 ,15, 0 ,1.3/4.5)
MAY_RPI = cal_RPI(8.4, 10, 0, 2.5/4.5)
JUNE_RPI = cal_RPI(8.5, 16, 0, 1.7/4.5)
JULY_RPI = cal_RPI(6.4, 24, 0, 2.6/4.5)
AUGUST_RPI = cal_RPI(8.1, 143, 0, 1.2/4.5)
SEPTEMBER_RPI = cal_RPI(7.7, 53, 0, 1.5/4.5)
OCTOBER_RPI = cal_RPI(7.8, 23, 0, 1.2 / 4.5)

In [6]:
# Printing the calculated values
print("April RPI:", APRIL_RPI)
print("May RPI:", MAY_RPI)
print("June RPI:", JUNE_RPI)
print("July RPI:", JULY_RPI)
print("August RPI:", AUGUST_RPI)
print("September RPI:", SEPTEMBER_RPI)
print("October PRI:", OCTOBER_RPI)

April RPI: 1.0
May RPI: 1.5
June RPI: 1.0
July RPI: 2.5
August RPI: 3.25
September RPI: 2.25
October PRI: 1.5


In [7]:
# todo: change the label
def get_label(val):
    if val <= 2.0:
        return "NP"
    elif val <= 3.0:
        return "SP"
    elif val <= 6.0:
        return "MP"
    else:
        return "SEVERE"

In [8]:
folder_paths = ['./水質檢測/Image_data/1_April',
                './水質檢測/Image_data/2_May',
                './水質檢測/Image_data/3_June',
                './水質檢測/Image_data/4_July',
                './水質檢測/Image_data/5_August',
                './水質檢測/Image_data/6_September']
labels = [get_label(APRIL_RPI), get_label(MAY_RPI), get_label(JUNE_RPI), 
          get_label(JULY_RPI), get_label(AUGUST_RPI), get_label(SEPTEMBER_RPI)]  # 对应文件夹1和文件夹2的标签


oct_paths = ["./水質檢測/Image_data/7_October"]
oct_label_Stirng = [get_label(OCTOBER_RPI)]

In [9]:
import tensorflow as tf
import os

def load_and_label_images(folder_path, label):
    image_paths = []  # 存储图像文件的路径
    labels = []       # 存储图像对应的标签

    # 获取文件夹中的所有图像文件的路径
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # 假设只加载jpg和png格式的图像文件
            image_paths.append(os.path.join(folder_path, filename))
            labels.append(label)

    # 加载图像并为其指定标签
    images = [tf.io.read_file(image_path) for image_path in image_paths]
    images = [tf.image.decode_image(image, channels=3) for image in images]

    # 可选的数据预处理：这里假设对图像进行归一化
    #images = [tf.cast(image, tf.float32) / 255.0 for image in images]

    return images, labels

In [10]:
# 你可以根据需要加载多个文件夹中的图像并为其指定不同的标签
# folder_paths = ['/path/to/folder1', '/path/to/folder2']
# labels = [1, 2]  # 对应文件夹1和文件夹2的标签

# 加载所有文件夹中的图像和标签
all_images = []
all_labels = []
for folder_path, label in zip(folder_paths, labels):
    images, labels = load_and_label_images(folder_path, label)
    all_images.extend(images)
    all_labels.extend(labels)
    

oct_images = []
oct_labels = []
for oct_path, label in zip(oct_paths, oct_label_Stirng):
    images, labels = load_and_label_images(oct_path, oct_label_Stirng)
    images_resized = [tf.image.resize(img, (224, 224)) for img  in images]
    oct_images.extend(images_resized)
    oct_labels.extend(labels)

In [11]:
# print(all_labels)
# print(oct_labels)

label_mapping = {
    "NP":[1,0,0],
    "MP":[0,1,0],
    "SP":[0,0,1]
}

# unique_labels_list = list(set(all_labels))
# unique_labels_list.sort()
# label_to_index = {label: index for index, label in enumerate(unique_labels_list)}
# label_one_hot = [tf.one_hot(label_to_index[label], len(unique_labels_list)) for label in all_labels]
label_one_hot = [label_mapping[label] for label in all_labels]
oct_label_one_hot = [label_mapping[label[0]] for label in oct_labels]


In [12]:
print(len(all_images))
print(all_images[0].shape)
print(len(label_one_hot))
print(label_one_hot[0:10])

print("=============================")

print(len(oct_images))
print(oct_images[0].shape)
print(len(oct_label_one_hot))
print(oct_label_one_hot[0:10])

9728
(224, 224, 3)
9728
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]]
612
(224, 224, 3)
612
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]]


In [13]:

# train_images, validation_images, train_labels, validation_labels = train_test_split(all_images, label_one_hot, test_size=0.2, random_state=42)

# # 创建训练集数据集
# train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size=32)

# # 创建验证集数据集
# validation_dataset = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))
# validation_dataset = validation_dataset.shuffle(buffer_size=len(validation_images)).batch(batch_size=32)

# 創建測試資料集
test_dataset = tf.data.Dataset.from_tensor_slices((oct_images, oct_label_one_hot))
test_dataset = test_dataset.shuffle(buffer_size=len(oct_images)).batch(batch_size=32)


In [14]:
# # 构建数据集
# dataset = tf.data.Dataset.from_tensor_slices((all_images, label_one_hot))

# # 可选的打乱和分批处理
# dataset = dataset.shuffle(buffer_size=len(all_images)).batch(batch_size=32)

In [15]:
# import matplotlib.pyplot as plt

# # 从数据集中获取并显示图像
# fig, axs = plt.subplots(2, 5, figsize=(20, 8))  # 创建一个2x5的子图布局

# for i, (image, label) in enumerate(dataset.take(10)):  # 取出前十张图像
#     row = i // 5  # 计算当前图像应该位于的行索引
#     col = i % 5   # 计算当前图像应该位于的列索引

#     axs[row, col].imshow(image[0])  # 假设每个batch里只有一张图像
#     axs[row, col].set_title('Label: {}'.format(label[0].numpy()))
#     axs[row, col].axis('off')  # 关闭坐标轴

# plt.show()

In [16]:

# 定义一个简单的卷积神经网络模型
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Dropout(0.1), 
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dropout(0.1),   
    tf.keras.layers.Dense(3, activation='softmax')
])
    return model

In [17]:
num_fold = 10
all_images = np.array(all_images)
label_one_hot = np.array(label_one_hot)

kf = KFold(n_splits=num_fold, shuffle=True, random_state=42)
fold = 1
confusion_matrices = []

for train_index, val_index in kf.split(all_images):
    print(f"Training fold {fold}...")
    # print(f"train_index={train_index}, val_index={val_index}")
    fold += 1
    
    train_images, val_images = all_images[train_index], all_images[val_index]
    train_labels, val_labels = label_one_hot[train_index], label_one_hot[val_index]
    
    # 創建數據集
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(32)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
    val_dataset = val_dataset.batch(32)
    
    # 創建和編譯模型
    model = create_model()
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    
    # 訓練模型
    model.fit(train_dataset, validation_data=val_dataset, epochs=2, callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath=f'best_model_fold_{fold}.keras', 
                                           monitor='val_accuracy', 
                                           save_best_only=True, 
                                           mode='max', 
                                           verbose=1)
    ])
    
    # 評估模型
    model.load_weights(f'best_model_fold_{fold}.keras')
    
    # 使用模型預測
    predictions = model.predict(val_dataset)
    predicted_classes_index = np.argmax(predictions, axis=1)
    
    true_classes_index = np.argmax(val_labels, axis=1)
    
    # 計算混淆矩陣
    conf_matrix = confusion_matrix(true_classes_index, predicted_classes_index)
    confusion_matrices.append(conf_matrix)
    
    print("Confusion Matrix for fold {}:".format(fold))
    print(conf_matrix)
    model.save(f"model_fold_{fold}.h5")

# 平均混淆矩陣
average_conf_matrix = np.mean(confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(average_conf_matrix)

Training fold 1...
Epoch 1/2
274/274 [==============================] - ETA: 0s - loss: 18.7676 - accuracy: 0.5326
Epoch 1: val_accuracy improved from -inf to 0.52621, saving model to best_model_fold_2.keras
274/274 [==============================] - 373s 1s/step - loss: 18.7676 - accuracy: 0.5326 - val_loss: 0.9978 - val_accuracy: 0.5262
Epoch 2/2
274/274 [==============================] - ETA: 0s - loss: 1.0854 - accuracy: 0.5432
Epoch 2: val_accuracy improved from 0.52621 to 0.55190, saving model to best_model_fold_2.keras
31/31 [==============================] - 8s 255ms/step
Confusion Matrix for fold 2:
[[487   1  11]
 [171   5   2]
 [249   2  45]]
Training fold 2...
Epoch 1/2
274/274 [==============================] - ETA: 0s - loss: 12.0126 - accuracy: 0.7885
Epoch 1: val_accuracy improved from -inf to 0.99692, saving model to best_model_fold_3.keras
274/274 [==============================] - 393s 1s/step - loss: 12.0126 - accuracy: 0.7885 - val_loss: 0.0592 - val_accuracy: 0.99

In [18]:
# model = create_model()

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.categorical_crossentropy,
#               metrics=['accuracy'])

In [19]:
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath='best_model.keras',  # 保存模型的文件路径
#     monitor='val_accuracy',    # 监控的指标，可以是'val_loss'或'val_accuracy'
#     save_best_only=True,       # 仅保存最好的模型
#     mode='max',                # 对于准确率，使用'max'模式
#     verbose=1                  # 输出详细日志信息
# )

In [20]:
# 這裡建議把epoch設小一點
# 我跑的時候差不多一個epoch要5~7分鐘

# result = model.fit(train_dataset, 
#                    validation_data=validation_dataset, 
#                    epochs=5,verbose=1, 
#                    callbacks=[checkpoint_callback])

In [21]:
# plt.plot(result.history['accuracy'], label='accuracy')
# plt.plot(result.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0, 1])
# plt.legend(loc='lower right')
# plt.show()

In [22]:
# from time import datetime
# model_name = "model" + datetime.now().strftime("%Y-%m-%d") + ".h5"
# model_name = "model.h5"
# model.save(model_name)

In [39]:
# model_name=""
# model = tf.keras.models.load_model(model_name)

In [ ]:


# # 使用model預測
# predictions = model.predict(test_dataset)
# # 轉換成class index
# predicted_classes_index = np.argmax(predictions, axis=1)
# true_classes_index = np.argmax([label.numpy() for label in oct_label_one_hot], axis=1)

# # 計算confusion matrix
# conf_matrix = confusion_matrix(true_classes_index, predicted_classes_index)

# print("Confusion Matrix:")
# print(conf_matrix)

In [40]:
all_test_predictions = []

for fold in range(2, num_fold+2):
    model_name=f"model_fold_{fold}.h5"
    model = tf.keras.models.load_model(model_name)
    fold_predictions = model.predict(test_dataset)
    all_test_predictions.append(fold_predictions)
    
average_test_predictions = np.mean(all_test_predictions, axis=0)
predicted_classes_index = np.argmax(average_test_predictions, axis=1)

20/20 [==============================] - 5s 265ms/step


In [42]:
# 所有模型的預測結果的平均



true_classes_index = np.argmax([np.array(label) for label in oct_label_one_hot], axis=1)
# print(true_classes_index)

# print(f"({len(all_test_predictions)}, {len(all_test_predictions[0])}, {len(all_test_predictions[0][0])})")
# print(average_test_predictions.shape)
# print(predicted_classes_index.shape)
# print(average_test_predictions)

# # 計算confusion matrix
conf_matrix = confusion_matrix(true_classes_index, predicted_classes_index, labels=np.arange(3))

print("Confusion Matrix:")
print(conf_matrix)

# 计算准确率
accuracy = accuracy_score(true_classes_index, predicted_classes_index)
print("Accuracy:", accuracy)

Confusion Matrix:
[[611   0   1]
 [  0   0   0]
 [  0   0   0]]
Accuracy: 0.9983660130718954


In [ ]:
# from sklearn.metrics import roc_auc_score

# predictions = np.array(predictions)
# true_labels = np.array(oct_label_one_hot)
# print(predictions.shape, true_labels.shape)

# # 計算AUCROC
# aucroc = roc_auc_score(true_labels, predictions, average='macro', multi_class='ovo')
# print("AUCROC:", aucroc)